In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("final_data.csv")
data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)

In [4]:
delta=df[['u_id', 'a_id', 'score']]
delta

,u_id,a_id,score
0,0,12403,6
1,104,12403,9
2,109,12403,8
3,460,12403,7
4,1336,12403,10
...,...,...,...
59149,15121,3230,8
59150,15698,3230,8
59151,22915,3230,7
59152,24855,3230,6


In [5]:
data_matrix

a_id,1,5,6,7,15,16,19,20,21,22,...,39417,39456,39468,39491,39523,39533,39539,39597,39799,40004
u_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
data_matrix.columns

Index([    1,     5,     6,     7,    15,    16,    19,    20,    21,    22,
       ...
       39417, 39456, 39468, 39491, 39523, 39533, 39539, 39597, 39799, 40004],
      dtype='int64', name='a_id', length=1590)

In [6]:
user_id_to_index = {} #helper function
for idx in range(len(data_matrix.index)):
    u_id = data_matrix.index[idx]
    user_id_to_index[u_id] = idx
    
user_id_to_index[104]

70

In [12]:
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx

anime_id_to_index[12403]

782

In [8]:
user_id_to_index[1892]

999

In [10]:
def baseline_model(x):

    delta = df[['u_id', 'a_id', 'score']]
    avg_rating = df['score'].mean()
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    b_u=cp.Variable(num_users)
    b_i=cp.Variable(num_anime)

    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx
        
    R_ui=[]
    for ui in range(len(delta)):
        u = user_id_to_index[int(delta.at[ui,'u_id'])]
        i = anime_id_to_index[int(delta.at[ui,'a_id'])]
        R_ui.append((avg_rating + b_u[u] + b_i[i])) #R_ui=avg_rating+b_u[u]+b_i[i]

    training_error=[]
    for ui in range(len(delta)):
        training_error.append((R_ui[ui]-int(delta.at[ui,'score']))**2) #(R_ui-A_ui)^2

    training_error_sum=cp.sum(training_error) #sum((R_ui-A_ui)^2)

    regularized_u=cp.sum_squares(b_u) #sum(b_u^2)
    regularized_i=cp.sum_squares(b_i) #sum(b_i^2)

    obj=cp.Minimize(training_error_sum+ x*(regularized_u+regularized_i))
    prob=cp.Problem(obj)
    prob.solve()
    print("status:", prob.status)
    
    R = np.zeros((num_users, num_anime))
    for u in range(num_users):
        for i in range(num_anime):
            R[u, i] = avg_rating + b_u.value[u] + b_i.value[i]
    
    print(R[70][782])

    



In [11]:
baseline_model(1000)


C:\Users\pakch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


status: optimal
7.421626205192989
